In [13]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
import re

import spacy
from spacy.matcher import Matcher

In [14]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(file, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                laparams=LAParams())

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter)

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

# calling above function and extracting text
text = ''
for page in extract_text_from_pdf('Чухров Александр.pdf'):
    text +=  page

text



'Чухров Александр\n\nМужчина, 26\xa0лет, родился 15 июня 1997\n+7\xa0(931)\xa05337618\nkurbobo239@gmail.com\xa0—\xa0предпочитаемый способ связи\nLinkedIn: https://www.linkedin.com/in/%D0%B0%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%\nBD%D0%B4%D1%80-%D1%87%D1%83%D1%85%D1%80%D0%BE%D0%B2-376a201a0/\nПроживает: Санкт-Петербург, м.\xa0Площадь Мужества\nГражданство: Россия,\xa0есть разрешение на работу: Россия\nНе готов к переезду, готов к редким командировкам\n\nЖелаемая должность и зарплата\nML инженер\nСпециализации:\n—  Программист, разработчик\nЗанятость: полная занятость\nГрафик работы: гибкий график, удаленная работа\nЖелательное время в пути до работы: не более полутора часов\nОпыт работы — 5 лет 1 месяц\nАпрель 2022\xa0—\nнастоящее время\n1 год 4 месяца\n\nTevian (ООО Технологии видеоанализа)\nML engineer\nЗанимаюсь распознаванием документов, а именно text detection, form understanding.\nРаботаю на pytorch с моделями типа transformer, применительно к CV\nООО "Ситимап"\nML engineer\nРазработк

In [8]:
# load pre-trained model
model_nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(model_nlp.vocab)

def extract_name(resume_text):
    nlp_text = model_nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text
    
extract_name(text)

'Чухров Александр'

In [20]:
def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number
        
extract_mobile_number(text)

'+79315337618'

In [21]:
def extract_email(text):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", text)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None
        
extract_email(text)

'kurbobo239@gmail.com'

In [ ]:
y = 'huy'
x = (
    f"{y}fdfd, "
    f"{y}, "
)

x